In [ ]:
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task

import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [ ]:
task = Task('binary',
            loss='logloss',
            metric='auc')
N_JOBS = 4
N_FOLDS = 4
RAM_LIMIT = 12
N_THREADS = N_JOBS
RANDOM_STATE = 42
TIMEOUT = 2400 # seconds
TARGET_NAME = 'target'

np.random.seed(RANDOM_STATE)

roles = {
    'target': TARGET_NAME,
    'drop': ['Статус брони']
}

In [ ]:
%%time

automl = TabularUtilizedAutoML(task = task, 
                       timeout = TIMEOUT,
                       cpu_limit = N_THREADS,
                       general_params = {'use_algos': [['lgb', 'xgb', 'lgb_tuned','linear_l2', 'xgb_tuned', 'cb', 'cb_tuned']]},
                       reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS},
                       tuning_params = {'max_tuning_time': 1200})

In [ ]:
predictions_oof = automl.fit_predict(train, 
                   roles=roles,
                   valid_data=Test,
                   verbose=1)

In [ ]:
# saving oof predictions
pd.Series(predictions_oof).to_csv('automl_predsoof_train.csv', index = False, header = False)

In [ ]:
# saving final predictions
predictions = automl.predict(test).data.flatten().round()
pd.Series(predictions).to_csv('automl_preds_final.csv', index = False, header = False)
pd.Series(predictions.data.flatten()).to_csv('automl_predsoof.csv', index = False, header = False)